In [ ]:
import numpy as np
 
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model # This is the functional API
 
vgg = VGG16(weights='imagenet', include_top=False) # 모델에서 Fully Connected layer 부분을 제거해주는 역할

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

base_dir = '/content/drive/MyDrive/apple'

train_dir = os.path.join(base_dir, 'train')
train_fresh_dir = os.path.join(train_dir, 'fresh')
train_rotten_dir = os.path.join(train_dir, 'rotten')

test_apple_dir = os.path.join(base_dir, 'test_apple')
test_apple_fresh_dir = os.path.join(test_apple_dir, 'fresh')
test_apple_rotten_dir = os.path.join(test_apple_dir, 'rotten')

test_banana_dir = os.path.join(base_dir, 'test_banana')
test_banana_fresh_dir = os.path.join(test_banana_dir, 'fresh')
test_banana_rotten_dir = os.path.join(test_banana_dir, 'rotten')

test_orange_dir = os.path.join(base_dir, 'test_orange')
test_orange_fresh_dir = os.path.join(test_orange_dir, 'fresh')
test_orange_rotten_dir = os.path.join(test_orange_dir, 'rotten')

In [ ]:
train_fresh_fnames = os.listdir(train_fresh_dir)
train_fresh_fnames.sort()
print(train_fresh_fnames[:10])

train_rotten_fnames = os.listdir(train_rotten_dir)
train_rotten_fnames.sort()
print(train_rotten_fnames[:10])

['Screen Shot 2018-06-08 at 4.59.36 PM.png', 'Screen Shot 2018-06-08 at 4.59.49 PM.png', 'Screen Shot 2018-06-08 at 4.59.57 PM.png', 'Screen Shot 2018-06-08 at 5.00.03 PM.png', 'Screen Shot 2018-06-08 at 5.00.12 PM.png', 'Screen Shot 2018-06-08 at 5.00.18 PM.png', 'Screen Shot 2018-06-08 at 5.00.26 PM.png', 'Screen Shot 2018-06-08 at 5.00.35 PM.png', 'Screen Shot 2018-06-08 at 5.00.43 PM.png', 'Screen Shot 2018-06-08 at 5.00.50 PM.png']
['Screen Shot 2018-06-07 at 2.15.20 PM.png', 'Screen Shot 2018-06-07 at 2.15.50 PM.png', 'Screen Shot 2018-06-07 at 2.16.18 PM.png', 'Screen Shot 2018-06-07 at 2.16.41 PM.png', 'Screen Shot 2018-06-07 at 2.17.15 PM.png', 'Screen Shot 2018-06-07 at 2.17.25 PM.png', 'Screen Shot 2018-06-07 at 2.18.13 PM.png', 'Screen Shot 2018-06-07 at 2.18.57 PM.png', 'Screen Shot 2018-06-07 at 2.19.15 PM.png', 'Screen Shot 2018-06-07 at 2.19.37 PM.png']


In [ ]:
print('total training fresh images:', len(os.listdir(train_fresh_dir)))
print('total training rotten images:', len(os.listdir(train_rotten_dir)))
print('total test_apple fresh images:', len(os.listdir(test_apple_fresh_dir)))
print('total test_apple rotten images:', len(os.listdir(test_apple_rotten_dir)))
print('total test_banana fresh images:', len(os.listdir(test_banana_fresh_dir)))
print('total test_banana rotten images:', len(os.listdir(test_banana_rotten_dir)))
print('total test_orange fresh images:', len(os.listdir(test_orange_fresh_dir)))
print('total test_orange rotten images:', len(os.listdir(test_orange_rotten_dir)))

total training fresh images: 1693
total training rotten images: 2342
total test_apple fresh images: 395
total test_apple rotten images: 601
total test_banana fresh images: 381
total test_banana rotten images: 530
total test_orange fresh images: 388
total test_orange rotten images: 403


In [ ]:
from tensorflow.keras import layers
img_input = layers.Input(shape=(224*224*3))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_apple_datagen = ImageDataGenerator(rescale=1./255)
test_banana_datagen = ImageDataGenerator(rescale=1./255)
test_orange_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh','rotten'],
        class_mode="binary")

test_apple_generator = test_apple_datagen.flow_from_directory(
        test_apple_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


test_banana_generator = test_banana_datagen.flow_from_directory(
        test_banana_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")

test_orange_generator = test_orange_datagen.flow_from_directory(
        test_orange_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


# 전체데이터 : 배치사이즈 수정 필요
# resnet50 input image size ; 224 x 224 x 3

Found 4035 images belonging to 2 classes.
Found 996 images belonging to 2 classes.
Found 911 images belonging to 2 classes.
Found 791 images belonging to 2 classes.


In [ ]:
for layer in vgg.layers:
    layer.trainable = False
# ResNet50 레이어들의 파라미터를 학습하지 않도록 설정합니다.
# 이렇게 설정된 매개 변수는 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트 되지 않습니다.

In [ ]:
x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(vgg.input, predictions)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
      train_generator, 
      epochs=30,
      verbose=1)

Epoch 1/30
16/16 [==============================] - 443s 29s/step - loss: 0.5498 - accuracy: 0.7594
Epoch 2/30
16/16 [==============================] - 19s 1s/step - loss: 0.3768 - accuracy: 0.8644
Epoch 3/30
16/16 [==============================] - 19s 1s/step - loss: 0.2932 - accuracy: 0.8947
Epoch 4/30
16/16 [==============================] - 19s 1s/step - loss: 0.2479 - accuracy: 0.9090
Epoch 5/30
16/16 [==============================] - 19s 1s/step - loss: 0.2146 - accuracy: 0.9323
Epoch 6/30
16/16 [==============================] - 19s 1s/step - loss: 0.1912 - accuracy: 0.9383
Epoch 7/30
16/16 [==============================] - 19s 1s/step - loss: 0.1688 - accuracy: 0.9465
Epoch 8/30
16/16 [==============================] - 19s 1s/step - loss: 0.1536 - accuracy: 0.9554
Epoch 9/30
16/16 [==============================] - 19s 1s/step - loss: 0.1418 - accuracy: 0.9596
Epoch 10/30
16/16 [==============================] - 19s 1s/step - loss: 0.1363 - accuracy: 0.9579
Epoch 11/30
16/16

In [ ]:
loss_and_metrics_apple = model.evaluate(test_apple_generator)
print('loss_and_metrics_apple : ' + str(loss_and_metrics_apple))

4/4 [==============================] - 218s 72s/step - loss: 0.0555 - accuracy: 0.9849
loss_and_metrics_apple : [0.05548738315701485, 0.9849397540092468]


In [ ]:
loss_and_metrics_banana = model.evaluate(test_banana_generator)
print('loss_and_metrics_banana : ' + str(loss_and_metrics_banana))

4/4 [==============================] - 186s 62s/step - loss: 0.4837 - accuracy: 0.8397
loss_and_metrics_banana : [0.48366880416870117, 0.8397365808486938]


In [ ]:
loss_and_metrics_orange = model.evaluate(test_orange_generator)
print('loss_and_metrics_orange : ' + str(loss_and_metrics_orange))

4/4 [==============================] - 159s 52s/step - loss: 0.2970 - accuracy: 0.8824
loss_and_metrics_orange : [0.29697757959365845, 0.8824273347854614]


In [ ]:
# 모델 저장하기
from keras.models import load_model
model.save('apple_VGG16_Dense256_epoch30.h5')

# 2. 모델 불러오기
# from keras.models import load_model
# model = load_model('모델파일명.h5')